<a href="https://colab.research.google.com/github/Vishwaak/Time-for-Machine/blob/master/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch

num_entries = 5000
db = torch.rand(num_entries) > 0.5
db


tensor([0, 1, 0,  ..., 1, 0, 1], dtype=torch.uint8)

In [0]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [0]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs
  

In [0]:
pdbs = get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs


In [0]:


db, pdbs = create_db_and_parallels(5000)



In [0]:
def query(db):
  return db.sum()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [38]:
sensitivity

tensor(1)

In [0]:
def sensitivity(query, n_entries=1000):

    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
            
    return max_distance
    
    

In [0]:
def query(db):
    return db.float().mean()

In [41]:
sensitivity(query)

tensor(0.0005)

In [0]:
db, pdbs = create_db_and_parallels(20)

In [43]:
db


tensor([0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
       dtype=torch.uint8)

In [0]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()

In [45]:
for i in range(10):
    sens_f = sensitivity(query, n_entries=10)
    print(sens_f)


tensor(1.)
tensor(1.)
0
0
0
0
0
0
0
tensor(1.)


In [0]:

db, _  = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db , remove_index=10)

In [48]:
db[10]

tensor(0, dtype=torch.uint8)

In [49]:
sum(db)

tensor(54, dtype=torch.uint8)

In [50]:
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [51]:
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0055)

In [52]:
(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)

In [53]:
db 


tensor([0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 0], dtype=torch.uint8)

In [0]:
def query(db):
  true_result = torch.mean(db.float())
  
  first_coin_flip = (torch.rand(len(db)) < 0.5).float()
  second_coin_flip = (torch.rand(len(db)) < 0.5).float()
  augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
  db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
  return db_result, true_result


In [55]:

db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))



With Noise:tensor(0.1000)
Without Noise:tensor(0.5000)


In [58]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))


With Noise:tensor(0.4936)
Without Noise:tensor(0.4948)


In [0]:
def query(db,noise = 0.2):
  true_result = torch.mean(db.float())
  
  first_coin_flip = (torch.rand(len(db)) < noise).float()
  second_coin_flip = (torch.rand(len(db)) < 0.5).float()
  
  augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
  sk_result = augmented_database.float().mean()
  
  private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)
  
  return private_result , true_result
  

In [69]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db, noise=0.1)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))


With Noise:tensor(0.3889)
Without Noise:tensor(0.4000)


In [71]:
db , pdbs = create_db_and_parallels(100)
private_result , true_result = query(db ,noise=0.2)
print("With noise: " + str(private_result))
print("Without noise: " + str(true_result))

With noise: tensor(0.5875)
Without noise: tensor(0.4500)


In [72]:
db , pdbs = create_db_and_parallels(100)
private_result , true_result = query(db ,noise=0.4)
print("With noise: " + str(private_result))
print("Without noise: " + str(true_result))

With noise: tensor(0.5833)
Without noise: tensor(0.5700)
